In [ ]:
from fastai.tabular.all import *

In [ ]:
path = Path('../input/jane-street-market-prediction')

In [ ]:
df = pd.read_csv(path/'train.csv', nrows=100000)
df.head()

In [ ]:
cat_names = ['date', 'feature_0']

In [ ]:
cont_names = [f'feature_{i}' for i in range(1,130)]

In [ ]:
df=df[df['weight']!=0]
df['action'] = ((df['resp']>0)*1).astype('category')

In [ ]:
splits = RandomSplitter(valid_pct=0.1)(range_of(df))

In [ ]:
to = TabularPandas(df, procs=[Categorify, FillMissing, Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names= 'action',
                   splits=splits)

In [ ]:
dls = to.dataloaders(bs=1024)

In [ ]:
dls.show_batch()

In [ ]:
learn = tabular_learner(dls, metrics=[accuracy]).to_native_fp16()

In [ ]:
learn.lr_find()

In [ ]:
cbs = [GradientAccumulation(),
      EarlyStoppingCallback(monitor='accuracy', comp=np.greater, min_delta=0.01, patience=4), 
      SaveModelCallback(monitor='accuracy', comp=np.greater, min_delta=0.01),
      ReduceLROnPlateau(monitor='accuracy', comp=np.greater, min_delta=0.01, patience=2)]

In [ ]:
learn.fit_one_cycle(30, 1e-3, cbs=cbs)

In [ ]:
learn.load('model')

In [ ]:
learn.to_native_fp32()

In [ ]:
import janestreet
#test = pd.read_csv("../input/jane-street-market-prediction/example_test.csv")
env = janestreet.make_env() 
iter_test = env.iter_test()

In [ ]:
from tqdm import tqdm
for (test_df,sample_pred) in tqdm(iter_test):
    test = test_df.copy()
    test.fillna(0, inplace=True)
    dl = learn.dls.test_dl(test)
    output, _ = learn.get_preds(dl=dl)
    sample_pred['action'] = output.argmax(1)
    env.predict(sample_pred)